## <center> Mining non-functional requirements </center> 

In [64]:
%run imports.ipynb
%run utils.ipynb

In [65]:
data = pd.read_csv('bases/train.csv')

In [66]:
data['clean_text'] = data['content_review'].apply(process_text)

In [67]:
labels  = []

for i in data['label']:
    a = i.split(',')
    labels.append(a)

mlb = MultiLabelBinarizer(classes=("Dep", "Per", "Sup", "Usa", "Mis"))
labelsdf = pd.DataFrame(mlb.fit_transform(labels), columns=['Dep', 'Per', 'Sup', 'Usa', 'Mis']) 
data = data.assign(label_dep=labelsdf['Dep'].values, label_per=labelsdf['Per'].values, label_sup=labelsdf['Sup'].values, label_usa=labelsdf['Usa'].values, label_mis=labelsdf['Mis'].values)
data

,content_review,sentiment_analysis,domain_name,label,clean_text,label_dep,label_per,label_sup,label_usa,label_mis
0,"I love the app, I have used it for years The o...",Positive,PhotosVideos,Usa,"[love, app, use, year, problem, new, update, w...",0,0,0,1,0
1,It would be better if they actually released t...,Positive,PhotosVideos,Mis,"[good, actually, release, message, instead, ju...",0,0,0,0,1
2,"Needs to be an option to pay monthly, or to op...",Negative,PhotosVideos,Mis,"[need, option, pay, monthly, open, deleted, pi...",0,0,0,0,1
3,Please let there be a one week free trial I wo...,Positive,PhotosVideos,Mis,"[let, week, free, trial, premium, like, try]",0,0,0,0,1
4,Can\'t connect to server. Failed big time. Ple...,Negative,PhotosVideos,Sup,"[t, connect, server, fail, big, time, fix, pro...",0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
5995,"I followed all the instructions airplane mode,...",Neutral,Lifestyle,Mis,"[follow, instruction, airplane, mode, listen, ...",0,0,0,0,1
5996,What a disappointment for 4.99 Ugh,Negative,Lifestyle,Mis,"[disappointment, ugh]",0,0,0,0,1
5997,I hate it ... this app does not work.,Negative,Lifestyle,Usa,"[hate, app, work]",0,0,0,1,0
5998,"Im at 28 weeks, and I was able to listen to my...",Positive,Lifestyle,Mis,"[im, week, able, listen, babys, heart, beat, j...",0,0,0,0,1


In [68]:
data_dep = data.loc[data['label_dep'] == True] 
data_dep = data_dep.drop(columns=['label_per', 'label_sup', 'label_usa', 'label_mis'])
text_dep = data_dep['clean_text'].tolist()

data_per = data.loc[data['label_per'] == True]
data_per = data_per.drop(columns=['label_dep', 'label_sup', 'label_usa', 'label_mis'])
text_per = data_per['clean_text'].tolist()

data_sup = data.loc[data['label_sup'] == True] 
data_sup = data_sup.drop(columns=['label_dep', 'label_per', 'label_usa', 'label_mis'])
text_sup = data_sup['clean_text'].tolist()

data_usa = data.loc[data['label_usa'] == True] 
data_usa = data_usa.drop(columns=['label_dep', 'label_per', 'label_sup', 'label_mis'])
text_usa = data_usa['clean_text'].tolist()

data_mis = data.loc[data['label_mis'] == True] 
data_mis = data_mis.drop(columns=['label_dep', 'label_per', 'label_sup', 'label_usa'])
text_mis = data_mis['clean_text'].tolist()

In [69]:
texts = data['clean_text'].astype('str')

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df = 2, max_df = .95)

bow_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df = 2, max_df = .95)

X = tfidf_vectorizer.fit_transform(texts)
# X = bow_vectorizer.fit_transform(texts)
#y = data.iloc[:, 5:9].values
y = data.iloc[:, 5:10].values

print(X.shape)
print(y.shape)

(6000, 10628)
(6000, 5)


In [70]:
lsa = TruncatedSVD(n_components=100, n_iter=10, random_state=3)

X = lsa.fit_transform(X)
X.shape

(6000, 100)

In [71]:
np.random.seed(42)
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size=0.2)
((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

(((4798, 100), (4798, 5)), ((1202, 100), (1202, 5)))

### Scikit-multilearn classifiers

##### 1. BRkNNaClassifier

In [72]:
from skmultilearn.adapt import BRkNNaClassifier

classifierBRkNNa = BRkNNaClassifier(k=3)
classifierBRkNNa.fit(X_train, y_train)
predictionBRkNNa = classifierBRkNNa.predict(X_test)

print("Accuracy = ", accuracy_score(y_test, predictionBRkNNa))
print("Precision = ", precision_score(y_test, predictionBRkNNa, average = 'macro'))
print("Recall = ", recall_score(y_test, predictionBRkNNa, average = 'macro'))
print("F1 Score = ", f1_score(y_test, predictionBRkNNa, average = 'macro'))
#print("Hamming score = ", hamming_score(y_test, predictionBRkNNa))
print("Hamming loss = ", hamming_loss(y_test, predictionBRkNNa))

Accuracy =  0.5965058236272879
Precision =  0.5663651155222909
Recall =  0.3208320143843859
F1 Score =  0.3668882407044985
Hamming loss =  0.14675540765391015


##### 2. MLkNN

In [73]:
from skmultilearn.adapt import MLkNN

classifierMLkNN = MLkNN(k=3)
classifierMLkNN.fit(X_train, y_train)
predictionMLkNN = classifierMLkNN.predict(X_test)

print("Accuracy = ", accuracy_score(y_test, predictionMLkNN))
print("Precision = ", precision_score(y_test, predictionMLkNN, average = 'macro'))
print("Recall = ", recall_score(y_test, predictionMLkNN, average = 'macro'))
print("F1 Score = ", f1_score(y_test, predictionBRkNNa, average = 'macro'))
# print("Hamming score = ", hamming_score(y_test, predictionMLkNN))
print("Hamming loss = ", hamming_loss(y_test, predictionMLkNN))

Accuracy =  0.5965058236272879
Precision =  0.5663651155222909
Recall =  0.3208320143843859
F1 Score =  0.3668882407044985
Hamming loss =  0.14675540765391015


##### 3. MLARAM

In [74]:
from skmultilearn.adapt import MLARAM

classifierMLARAM = MLARAM(threshold=0.05, vigilance=0.95)
classifierMLARAM.fit(X_train, y_train)
predictionMLARAM = classifierMLARAM.predict(X_test)

print("Accuracy = ", accuracy_score(y_test, predictionMLARAM))
print("Precision = ", precision_score(y_test, predictionMLARAM, average = 'macro'))
print("Recall = ", recall_score(y_test, predictionMLARAM, average = 'macro'))
print("F1 Score = ", f1_score(y_test, predictionMLARAM, average = 'macro'))
print("Hamming score = ", hamming_score(y_test, predictionMLARAM))
print("Hamming loss = ", hamming_loss(y_test, predictionMLARAM))

Accuracy =  0.5840266222961731
Precision =  0.4729822865306737
Recall =  0.405906395975566
F1 Score =  0.410052112404866
Hamming score =  0.6485718247365502
Hamming loss =  0.16356073211314476


##### 4. BinaryRelevance

In [47]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

classifierBRSVC = BinaryRelevance(classifier = SVC(), require_dense = [False, True])
classifierBRSVC.fit(X_train, y_train)
predictionBRSVC = classifierBRSVC.predict(X_test)

#print("Accuracy = ", accuracy_score(y_test, predictionBRSVC))
#print("Precision = ", precision_score(y_test, predictionBRSVC, average = 'macro'))
#print("Recall = ", recall_score(y_test, predictionBRSVC, average = 'macro'))
#print("F1 Score = ", f1_score(y_test, predictionBRSVC, average = 'macro'))
#print("Hamming score = ", hamming_score(y_test, predictionBRSVC))
#print("Hamming loss = ", hamming_loss(y_test, predictionBRSVC))

##### 5. ClassifierChain

In [48]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.svm import SVC

classifierCCSVC = ClassifierChain(classifier = SVC(), require_dense = [False, True])
classifierCCSVC.fit(X_train, y_train)
predictionCCSVC = classifierCCSVC.predict(X_test)


#print("Accuracy = ", accuracy_score(y_test, predictionCCSVC))
#print("Precision = ", precision_score(y_test, predictionCCSVC, average = 'macro'))
#print("Recall = ", recall_score(y_test, predictionCCSVC, average = 'macro'))
#print("F1 Score = ", f1_score(y_test, predictionCCSVC, average = 'macro'))
#print("Hamming score = ", hamming_score(y_test, predictionCCSVC))
#print("Hamming loss = ", hamming_loss(y_test, predictionCCSVC))

##### 6. LabelPowerset

In [49]:
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier

classifierCCRF = ClassifierChain(classifier = RandomForestClassifier(n_estimators=100), require_dense = [False, True])
classifierCCRF.fit(X_train, y_train)
predictionCCRF = classifierCCRF.predict(X_test)

#print("Accuracy = ", accuracy_score(y_test, predictionCCRF))
#print("Precision = ", precision_score(y_test, predictionCCRF, average = 'macro'))
#print("Recall = ", recall_score(y_test, predictionCCRF, average = 'macro'))
#print("F1 Score = ", f1_score(y_test, predictionCCRF, average = 'macro'))
#print("Hamming score = ", hamming_score(y_test, predictionCCRF))
#print("Hamming loss = ", hamming_loss(y_test, predictionCCRF))

##### 7. RakelD

In [75]:
from sklearn.naive_bayes import GaussianNB
from skmultilearn.ensemble import RakelD

classifierRD = RakelD(base_classifier=GaussianNB(), base_classifier_require_dense=[True, True], labelset_size=4)
classifierRD.fit(X_train, y_train)
predictionRD = classifierRD.predict(X_test)

print("Accuracy = ", accuracy_score(y_test, predictionRD))
print("Precision = ", precision_score(y_test, predictionRD, average = 'macro'))
print("Recall = ", recall_score(y_test, predictionRD, average = 'macro'))
print("F1 Score = ", f1_score(y_test, predictionRD, average = 'macro'))
#print("Hamming score = ", hamming_score(y_test, predictionRD))
print("Hamming loss = ", hamming_loss(y_test, predictionRD))

Accuracy =  0.5141430948419301
Precision =  0.38962680370830627
Recall =  0.45228345710065077
F1 Score =  0.4056263916218086
Hamming loss =  0.19600665557404326


##### 8. MajorityVotingClassifier

In [51]:
from skmultilearn.ensemble import MajorityVotingClassifier
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB

#classifierMV = MajorityVotingClassifier(classifier = ClassifierChain(classifier=GaussianNB()))
#classifierMV.fit(X_train, y_train)
#predictionsMV = classifierMV.predict(X_test)

#print("Accuracy = ", accuracy_score(y_test, predictionsMV))
#print("Precision = ", precision_score(y_test, predictionsMV, average = 'macro'))
#print("Recall = ", recall_score(y_test, predictionsMV, average = 'macro'))
#print("F1 Score = ", f1_score(y_test, predictionsMV, average = 'macro'))
#print("Hamming score = ", hamming_score(y_test, predictionsMV))
#print("Hamming loss = ", hamming_loss(y_test, predictionsMV))